In [1]:
import pandas as pd
import numpy as np

In [2]:
import json
with open('users_with_age_and_gender.json', 'r')  as f:
    features = json.load(f)

In [3]:
len(features.keys())

153178

In [4]:
edges = pd.read_csv('edges.csv')

In [5]:
edges

,Source,Target
0,365,989472
1,365,2425825
2,365,76866
3,365,259
4,365,14092932
...,...,...
1387543,289489256,4214123
1387544,50563461,3526026
1387545,483877,101579784
1387546,8947289,9240403


In [6]:
edges2 = edges[(edges.Source.isin(np.array(list(features.keys())).astype(int)))&(edges.Target.isin(np.array(list(features.keys())).astype(int)))]

In [7]:
edges2

,Source,Target
0,365,989472
1,365,2425825
2,365,76866
3,365,259
4,365,14092932
...,...,...
1387543,289489256,4214123
1387544,50563461,3526026
1387545,483877,101579784
1387546,8947289,9240403


In [8]:
old_indexes = np.array(list(features.keys())).astype(int)
new_indexes = range(len(old_indexes))
voc1 = dict(zip(old_indexes, new_indexes))
voc2 = dict(zip(new_indexes, old_indexes))

In [9]:
edges2.Source = edges2.Source.apply(lambda x: voc1[x])
edges2.Target = edges2.Target.apply(lambda x: voc1[x])

d:\mlp\venv\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
edges2

,Source,Target
0,146316,63767
1,146316,36687
2,146316,80200
3,146316,5
4,146316,139240
...,...,...
1387543,49116,68107
1387544,45172,148700
1387545,2056,105510
1387546,42589,12528


In [11]:
features2 = dict(zip(new_indexes, features.values()))

In [12]:
format_features = []
for i in list(features2.keys()):
    arr = features2[i][0].copy()
    arr.append(features2[i][1])
    arr.append(features2[i][2])
    format_features.append(arr)

In [13]:
len(format_features)

153178

In [14]:
all_pairs = edges2.groupby('Source').Target.apply(list)

In [15]:
def bfs(graph, start):
    path = []
    queue = [start]
    while queue:
        vertex = queue.pop(0)
        if (vertex not in path):
            path.append(vertex)
            queue.extend(set(graph[vertex]) - set(path))
#             if len(path)%1000==0:
#                 print(len(path))
    return path

In [16]:
path0 = np.load('path_0.npy')
path1 = np.load('path_1.npy')

In [17]:
import torch
from torch_geometric.data import Data

In [18]:
# def create_graph(seq, el_num):
#     element = seq[el_num]
#     new_seq = seq[:el_num]
#     edge_index = torch.tensor(np.array(edges2[(edges2.Source.isin(new_seq)) & (edges2.Target.isin(new_seq))]))
#     res = np.append(np.array([-1]),np.array(pd.Series(seq[:el_num]).isin(all_pairs[element]).astype(int)))
#     x = torch.tensor(format_features)
#     node_feature = format_features[element]
#     data = Data(x=x, edge_index=edge_index.t().contiguous())
#     nodes_in_graph = [format_features[x] for x in new_seq]
#     return(data, res, node_feature, nodes_in_graph)

In [19]:
from torch_geometric.utils import remove_isolated_nodes
def create_graph(seq, el_num):
    element = seq[el_num]
    new_seq = seq[:el_num]
    edge_index = torch.tensor(np.array(edges2[(edges2.Source.isin(new_seq)) & (edges2.Target.isin(new_seq))]))
    res = np.append(np.array([-1]),np.array(pd.Series(seq[:el_num]).isin(all_pairs[element]).astype(int)))
    q = remove_isolated_nodes(edge_index)
    edge_index = torch.tensor(q[0])
    temp = np.zeros(len(format_features))
    temp[:len(q[2])] = q[2]
    x = torch.tensor(format_features)[temp.astype(bool)]
    node_feature = format_features[element]
    data = Data(x=x, edge_index=edge_index.t().contiguous())
    nodes_in_graph = [format_features[x] for x in new_seq]
    
    return(data, res, node_feature, nodes_in_graph)

In [20]:
def create_batch(sequences, el_num):
    graphs = []
    adj = []
    features = []
    nodes_in_graph = []
    for i in sequences:
        info = create_graph(i, el_num)
        graphs.append(info[0])
        adj.append(info[1])
        features.append(info[2])
        nodes_in_graph.append(info[3])
    return(graphs, adj, features, torch.tensor(nodes_in_graph).to(device))

In [24]:
from torch_geometric.utils import remove_isolated_nodes
def create_graph(seq, el_num):
    element = seq[el_num]
    new_seq = seq[:el_num]
    sorted_new_seq = np.sort(new_seq)
    order = [np.where(new_seq==x)[0].item() for x in sorted_new_seq]
    res = np.array(pd.Series(seq[:el_num]).isin(all_pairs[element]).astype(int))
    edge_index = torch.tensor(np.array(edges2[(edges2.Source.isin(new_seq)) & (edges2.Target.isin(new_seq))]))
    q = remove_isolated_nodes(edge_index)
    edge_index = torch.tensor(q[0])
    temp = np.zeros(len(format_features))
    temp[:len(q[2])] = q[2]
    x = torch.tensor(format_features)[temp.astype(bool)]
    data = Data(x=x, edge_index=edge_index.t().contiguous())
    return(data, res, order)

In [31]:
def create_batch(sequences, el_num):
    graphs = []
    results = []
    orders = []
    for i in sequences:
        info = create_graph(i, el_num)
        graphs.append(info[0])
        results.append(info[1])
        orders.append(info[2])
    return(graphs, results, orders)

In [32]:
create_batch([path0, path1], 10)

([Data(edge_index=[2, 18], x=[10, 112]),
  Data(edge_index=[2, 24], x=[10, 112])],
 [array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
  array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
 [[0, 8, 6, 7, 3, 4, 5, 2, 1, 9], [0, 2, 1, 9, 4, 3, 5, 7, 6, 8]])

In [21]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.nn as nn
from torch_geometric.nn import avg_pool, global_add_pool
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data, DataLoader, NeighborSampler
from torch.nn import BatchNorm1d, Linear

In [25]:
class Net2(torch.nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = GCNConv(112, 250)
        self.conv2 = GCNConv(250, 250)
        self.graph_to_h0 = nn.Linear(362, 100)
        self.graph_to_c0 = nn.Linear(362, 100)
        self.lstm = nn.LSTM(112, 100, batch_first = True) 
        self.logits = nn.Linear(100, 2)
    def forward(self, data, node_features, adj):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_add_pool(x, data.batch)
        x = F.relu(x)
        x = torch.cat((x,torch.tensor(node_features, requires_grad=True)), 1)
        initial_cell = self.graph_to_c0(x)
        initial_hid = self.graph_to_h0(x)
#         inp = torch.unsqueeze(adj, 2).float()
        state = (initial_cell[None], initial_hid[None])
        lstm_out, state = self.lstm(adj, state)
        logits = self.logits(lstm_out)
        res = F.softmax(logits)
        return res

In [26]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Net2()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), 
                       lr=1e-2)

In [27]:
def computeLoss2(network, batch, el_num, batch_size):
    info = create_batch(batch, el_num)
    loader = DataLoader(info[0], batch_size=batch_size)
    now_caption = torch.tensor(info[1])[:,:-1]
    next_caption = torch.tensor(info[1])[:,1:].to(device)
    node_features = torch.tensor(info[2]).to(device)
    nodes = torch.tensor(info[3]).to(device)
    for i in loader:
        q = i
        q = q.to(device)
        preds = network.forward(q, node_features, nodes)
    loss = nn.CrossEntropyLoss()
    preds = torch.transpose(preds, 1, 2)
    output = loss(preds, next_caption.long())
    return(output)

In [30]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
num_epoch = 20
n_batches_per_epoch = 4
batch_size = 3
n_validation_batches = 3
train_losses = []
val_losses = []
for epoch in range(num_epoch):
    train_loss= 0
    val_loss = 0
    model.train(True)
    order = np.random.permutation(len(train))
    order2 = np.random.permutation(len(test))
    indexes = np.random.randint(1, 10000, n_batches_per_epoch)
    for _ in (indexes):
        optimizer.zero_grad()
        batch_indexes = order[0:0+batch_size]
        x_batch = train[batch_indexes]
        x_batch = list(np.array(x_batch))
        loss_value = computeLoss2(model, x_batch, _, batch_size)
        loss_value.backward()
        optimizer.step()
        train_loss += loss_value.item()
    train_loss /= (n_batches_per_epoch)
    print(train_loss)
#     train_losses.append(train_loss)
#     model.train(False)
#     for _ in range(n_validation_batches):
#         batch_indexes = order2[0:0+batch_size]
#         x_batch = list(np.array(x_batch))
#         loss_value = computeLoss2(model, x_batch, _, batch_size)
#         val_loss += loss_value.item()
#     val_loss /= (n_validation_batches)
#     val_losses.append(val_loss)
#     clear_output()
#     plt.figure(figsize = (10, 6))
#     plt.plot(train_losses, label = 'Train loss', color = 'blue')
#     plt.plot(val_losses, label = 'Val loss', color = 'orange')
#     plt.axhline(y=3.0, color='gray', linestyle='--', label = 'Target loss')
#     plt.legend()
#     plt.ylabel('Loss')
#     plt.show()

0.6931659877300262
0.6931395530700684
0.6931557953357697
0.6931522190570831
0.6931534856557846
0.6931499242782593
0.6931523680686951
0.6931585967540741
0.6931524127721786


RuntimeError: CUDA error: unspecified launch failure